<a href="https://colab.research.google.com/github/dnguyend/rayleigh_newton/blob/master/colab/GeneralizedHPairsEigenTensor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

$\newcommand{\cT}{\mathcal{T}}$
$\newcommand{\cB}{\mathcal{B}}$
$\newcommand{\C}{\mathbb{C}}$
$\newcommand{\R}{\mathbb{R}}$
# This workbook compute all eigentensor pairs by Rayleigh quotient iteration for a generalized Tensor eigenvalue problem when the two tensors have the same order
* Eigenpairs  
$$\cT(I, X^{[m-1]}) =\lambda \cB(I, X^{[m-1]}) $$
The square bracket means the number of times $X$ is repeated.
* $\cT$ is a $(m, n)$ tensor, $\cB$ is a $(m, n)$ tensor. Both with coefficients in $\R$. However $X$ could be in $\C$.
* H-eigenpair is when $\cB(X^{[m]})=\sum x_i^m$ if $X=[x_0,\cdots x_n]$.

* Use generalized Rayleigh quotient method.

* Consider the complex case, $X\in \C^n$, consider the unitary constraint $X^*X = 1$. The left inverse is $\frac{1}{|\cB(I, X^{[d-1]})|^2}\cB(I, X^{[d-1]})^*$. The Rayleigh quotient is
$$\frac{1}{|\cB(I, X^{[d-1]})|^2}\cB(I, X^{[d-1]})^*\cT(I, X^{[m-1]})$$

The eigenvalue count is $n(m-1)^{n-1}$ conjecturally
* In contrast to the workbook ZPairsEigentensor, since we want to test the eigen pair count, we do not stop after the count is reached, but keep running for sometime, showing we do not get more pairs after the count is reached.


# We confirm the eigenpairs in examples 4.3 and 4.10 of [Cui et. al (2014)] and check the eigen pair count.
CUI , C.-F., DAI , Y.-H. & NIE , J. (2014) All real eigenvalues of symmetric tensors. SIAM Journal on MatrixAnalysis and Applications, 35, 1582–1601.

# Some cells may be hidden. Please open in colab then unhide the cells


First, clone the project from github


In [1]:
!git clone https://github.com/dnguyend/rayleigh_newton

Cloning into 'rayleigh_newton'...
remote: Enumerating objects: 249, done.
remote: Counting objects: 100% (249/249), done.
remote: Compressing objects: 100% (118/118), done.
remote: Total 249 (delta 130), reused 244 (delta 125), pack-reused 0
Receiving objects: 100% (249/249), 14.73 MiB | 25.61 MiB/s, done.
Resolving deltas: 100% (130/130), done.


In [2]:
from __future__ import print_function
import numpy as np
import pandas as pd
import time
from types import SimpleNamespace

import sys
import scipy.linalg
from numpy import concatenate, tensordot, eye, zeros, zeros_like,\
    power, sqrt, exp, pi

from numpy.linalg import solve, inv, norm

import rayleigh_newton.core.utils as utils
from rayleigh_newton.core.eigen_tensor_solver import symmetric_tv_mode_product

In [3]:
def schur_form_B_tensor_rayleigh_orthogonal_same_order(
        T, B, max_itr, delta, x_init=None):
    """Schur form rayleigh chebyshev unitary
    T and x are complex. Constraint is x^H x = 1
    lbd is real
    """
    # get tensor dimensionality and order
    n_vec = T.shape
    m = len(n_vec)
    d = len(B.shape)
    if m != d:
      print("T and B must have the same order m=%d != d=%d" % (m, d))
      return
    n = T.shape[0]
    R = 1
    converge = False

    # if not given as input, randomly initialize
    if x_init is None:
        x_init = np.random.randn(n)
        x_init = x_init/norm(x_init)

    # init lambda_(k) and x_(k)
    x_k = x_init.copy()
    T_x_m_2 = symmetric_tv_mode_product(T, x_k, m-2)
    T_x_m_1 = T_x_m_2 @ x_k

    B_x_m_2 = symmetric_tv_mode_product(B, x_k, d-2)
    B_x_m_1 = B_x_m_2 @ x_k

    lbd = (B_x_m_1.T @ T_x_m_1)/norm(B_x_m_1)**2
    ctr = 0

    while (R > delta) and (ctr < max_itr):
        # compute T(I,I,x_k,...,x_k), T(I,x_k,...,x_k) and g(x_k)
        rhs = concatenate(
            [B_x_m_1.reshape(-1, 1), T_x_m_1.reshape(-1, 1)-lbd*B_x_m_1.reshape(-1, 1)], axis=1)

        # compute Hessian H(x_k)
        H = (m-1)*T_x_m_2-lbd*(d-1)*B_x_m_2
        lhs = solve(H, rhs)

        # fix eigenvector
        y = lhs[:, 0] * (
            np.sum((x_k * lhs[:, 1])) /
            np.sum((x_k * lhs[:, 0]))) - lhs[:, 1]
        # print('%f %f %f' % (norm(lhs[:, 0]), norm(lhs[:, 1]), norm(np.sum(x_k.conjugate()*y))))
        x_k_n = (x_k + y) / norm(x_k + y)

        # x_k_n = (x_k + y)/(np.linalg.norm(x_k + y))

        #  update residual and lbd
        R = norm(x_k-x_k_n)
        x_k = x_k_n

        T_x_m_2 = symmetric_tv_mode_product(T, x_k, m-2)
        T_x_m_1 = T_x_m_2 @ x_k
        B_x_m_2 = symmetric_tv_mode_product(B, x_k, d-2)
        B_x_m_1 = B_x_m_2 @ x_k

        lbd = (B_x_m_1.T @ T_x_m_1)/norm(B_x_m_1)**2
        # print('ctr=%d lbd=%f' % (ctr, lbd))
        ctr += 1
    x = x_k
    err = norm(symmetric_tv_mode_product(
        T, x, m-1) - lbd * symmetric_tv_mode_product(
        B, x, d-1))
    if ctr < max_itr:
        converge = True

    return x, lbd, ctr, converge, err


def schur_form_B_tensor_rayleigh_unitary_same_order(
        T, B, max_itr, delta, x_init=None):
    """Schur form rayleigh chebyshev unitary
    T and x are complex. Constraint is x^H x = 1
    lbd is real
    """
    # get tensor dimensionality and order
    n_vec = T.shape
    m = len(n_vec)
    d = len(B.shape)
    if m != d:
      print("T and B must have the same order m=%d != d=%d" % (m, d))
      return
    n = T.shape[0]
    R = 1
    converge = False

    # if not given as input, randomly initialize
    if x_init is None:
        x_init = np.random.randn(n) + 1j*np.random.randn(n)
        x_init = x_init/norm(x_init)

    # init lambda_(k) and x_(k)
    x_k = x_init.copy()
    T_x_m_2 = symmetric_tv_mode_product(T, x_k, m-2)
    T_x_m_1 = T_x_m_2 @ x_k

    B_x_m_2 = symmetric_tv_mode_product(B, x_k, d-2)
    B_x_m_1 = B_x_m_2 @ x_k

    lbd = (B_x_m_1.conjugate().T @ T_x_m_1)/norm(B_x_m_1)**2
    ctr = 0

    while (R > delta) and (ctr < max_itr):
        # compute T(I,I,x_k,...,x_k), T(I,x_k,...,x_k) and g(x_k)
        rhs = concatenate(
            [B_x_m_1.reshape(-1, 1), T_x_m_1.reshape(-1, 1)-lbd*B_x_m_1.reshape(-1, 1)], axis=1)

        # compute Hessian H(x_k)
        H = (m-1)*T_x_m_2-lbd*(d-1)*B_x_m_2
        lhs = solve(H, rhs)

        # fix eigenvector
        y = lhs[:, 0] * (
            np.sum((x_k.conjugate() * lhs[:, 1])) /
            np.sum((x_k.conjugate() * lhs[:, 0]))) - lhs[:, 1]
        # print('%f %f %f' % (norm(lhs[:, 0]), norm(lhs[:, 1]), norm(np.sum(x_k.conjugate()*y))))
        x_k_n = (x_k + y) / norm(x_k + y)

        # x_k_n = (x_k + y)/(np.linalg.norm(x_k + y))

        #  update residual and lbd
        R = norm(x_k-x_k_n)
        x_k = x_k_n

        T_x_m_2 = symmetric_tv_mode_product(T, x_k, m-2)
        T_x_m_1 = T_x_m_2 @ x_k
        B_x_m_2 = symmetric_tv_mode_product(B, x_k, d-2)
        B_x_m_1 = B_x_m_2 @ x_k

        lbd = (B_x_m_1.conjugate().T @ T_x_m_1)/norm(B_x_m_1)**2
        # print('ctr=%d lbd=%f' % (ctr, lbd))
        ctr += 1
    x = x_k
    err = norm(symmetric_tv_mode_product(
        T, x, m-1) - lbd * symmetric_tv_mode_product(
        B, x, d-1))
    if ctr < max_itr:
        converge = True

    return x, lbd, ctr, converge, err

def complex_eigen_cnt(n, m, d):
    if m == d:
        return n*power(m-1, n-1)
    return (power(m-1, n)-power(d-1, n)) // (m-d)


def find_eig_cnt(all_eig):
    first_nan = np.where(np.isnan(all_eig.x))[0]
    if first_nan.shape[0] == 0:
        return None
    else:
        return first_nan[0]

    
def normalize_real(lbd, x, m, d, tol):
    """ First try to make it to a real pair
    if not possible. If not then make lambda real
    return is_self_conj, is_real, new_lbd, new_x
    """
    u = (sqrt(x @ x).conjugate())
    new_x = x * u
    # if np.sum(np.abs(new_x.imag)) < tol:
    if np.abs(np.abs(u) - 1) < tol:
        # try to flip. if u **(m-d) > 0 use it:
        # lbd_factor = lbd_factor.real
        new_x = (new_x.real + 0.j)/norm(new_x.real)
        return True, lbd, new_x

    return False, lbd, x


def _insert_eigen(all_eig, x, lbd, eig_cnt, m, d, tol, disc):
    """
    force eigen values to be positive if possible
    if x is not similar to a vector in all_eig.x
    then:
       insert pair x, conj(x) if x is not self conjugate
       otherwise insert x
    all_eig has a structure: lbd, x, is_self_conj, is_real
    """
    is_real, norm_lbd, norm_x = normalize_real(
        lbd, x, m, d, tol)

    if is_real:
        good_x = [norm_x]
        good_lbd = [norm_lbd]
    else:
        good_x = [norm_x, norm_x.conjugate()]
        good_lbd = [norm_lbd, norm_lbd.conjugate()]
    nct = 0
    for xx in good_x:
      #  factors = all_eig.x[:eig_cnt+nct, :] @ xx.conjugate()
      #fidx = np.where(np.abs(factors ** (m-2) - 1) < disc)[0]
      factors = all_eig.x[:eig_cnt+nct, :] @ xx.conjugate()
      fidx = np.where(np.abs(np.abs(factors)-1) < disc)[0]
      if fidx.shape[0] == 0:
        all_eig.lbd[eig_cnt+nct] = norm_lbd
        all_eig.x[eig_cnt+nct] = xx
        all_eig.is_self_conj[eig_cnt+nct] = False
        all_eig.is_real[eig_cnt+nct] = is_real
        nct += 1

    return eig_cnt + nct


def find_all_unitary_eigenpair_same_order(
        all_eig, eig_cnt, A, B, max_itr, max_test, tol, disc):
    """ output is the table of results
     2n*+2 columns: lbd, is self conjugate, x_real, x_imag
    This is the raw version, since the output vector x
    is not yet normalized to be real when possible
    """
    n = A.shape[0]
    m = len(A.shape)
    d = len(B.shape)
    if m != d:
      print("cannot deal with m=%d != n=%d" % (m, d))
      return None, None
    # n_eig = complex_eigen_cnt(n, m)
    n_eig = all_eig.lbd.shape[0]
    if all_eig is None:
        all_eig = SimpleNamespace(
            lbd=np.full((n_eig), np.nan, dtype=complex),
            x=np.full((n_eig, n), np.nan, dtype=complex),
            is_self_conj=zeros((n_eig), dtype=bool),
            is_real=zeros((n_eig), dtype=bool))
        eig_cnt = 0
    elif eig_cnt is None:
        eig_cnt = find_eig_cnt(all_eig)
        if eig_cnt is None:
            return all_eig

    for jj in range(max_test):
        x0r = np.random.randn(2*n-1)
        # x0r = np.random.randint(-10, 10, 2*n-1)*1.
        x0r /= norm(x0r)
        x0 = x0r[:n] + 0.j
        x0[1:] = x0[1:] + x0r[n:] * 1.j
        # x0[-1] = np.abs(x0[-1])
        # if there are odd numbers left,
        # try to find a real root
        draw = np.random.uniform(0, 1, 1)
        # 50% try real root
        if True and (draw < .5) and ((n_eig - eig_cnt) % 2 == 1):
            try:
                x_r, lbd, ctr, converge, err = schur_form_B_tensor_rayleigh_unitary_same_order(
                    A, B, max_itr, tol, x_init=x0.real)
                x = x_r + 1j * zeros((x_r.shape[0]))
            except Exception as e:
                print(e)
                continue
        else:
            try:
                x, lbd, ctr, converge, err =\
                    schur_form_B_tensor_rayleigh_unitary_same_order(
                        A, B, max_itr, tol, x_init=x0)
                # print(x0, x, lbd, ctr, converge, err)
            except Exception as e:
                print(e)
                continue
        old_eig = eig_cnt        
        if (err < tol):
            eig_cnt = _insert_eigen(all_eig, x, lbd, eig_cnt, m, d, tol, disc)
        if eig_cnt == n_eig:
            break
        # elif (eig_cnt > old_eig) and (eig_cnt % 10 == 0):
        elif (eig_cnt > old_eig) and True:
            print('Found %d eigenpairs' % eig_cnt)
    return SimpleNamespace(
            lbd=all_eig.lbd[:eig_cnt],
            x=all_eig.x[:eig_cnt, :],
            is_self_conj=all_eig.is_self_conj[:eig_cnt],
            is_real=all_eig.is_real[:eig_cnt]), eig_cnt
    
def save_out(T, B, all_eig, save_file):
    np.savez_compressed('%s_%d_%d_%d.npz' % (
        save_file, n, m, d), T=T, B=B, lbd=all_eig.lbd,
        x=all_eig.x, is_real=all_eig.is_real,
        is_self_conj=all_eig.is_self_conj)



A simple test. Generate a random tensor then run.

In [4]:
import numpy as np
import numpy.linalg as la

n = 5
m = 6

# np.random.seed(0)
T = utils.generate_symmetric_tensor(n, m)
B = utils.generate_symmetric_tensor(n, m)
max_itr = 200


x, lbd, cnt, converge, err = schur_form_B_tensor_rayleigh_unitary_same_order(
        T, B, max_itr, delta=1e-8, x_init=None)

print(x, lbd, cnt, converge, err)



[-0.01336904+0.23785311j -0.44082947-0.00220531j  0.35883612+0.11696456j
 -0.29300916-0.1413673j  -0.62521734+0.33125793j] (0.6334306545933598-0.154736749503515j) 8 True 1.0539812104219684e-15


Some code to beautify outputs

In [5]:
def check_eig(T, B, all_eig):
  max_run = all_eig.lbd.shape[0]
  # max_run = 76
  diff0 = np.empty(max_run)
  diff1 = np.empty(max_run)
  for i in range(max_run):
    Tnew = symmetric_tv_mode_product(T, all_eig.x[i, :], m-1)
    Bnew = symmetric_tv_mode_product(B, all_eig.x[i, :], m-1)
    lbdnew = np.sum(Bnew.conjugate()*Tnew)/norm(Bnew)**2
    diff0[i] = lbdnew - all_eig.lbd[i]
    diff1[i] = norm(Tnew - lbdnew*Bnew)
  print('check lbd %f' % np.max(np.abs(diff0)))
  print('check equation %f' % np.max(np.abs(diff1)))

  diff3 = np.empty(max_run-1)
  for i in range(max_run-1):
      factors = np.sum(all_eig.x[i, :].conj()*all_eig.x[i+1, :])
      diff3[i] = np.abs(np.abs(factors)-1)
  # print(np.argsort(diff3))
  print("check uniqueness")
  print(np.sort(diff3[np.where(diff3< 1e-1)]))

def display_one(comb, i):
  return (comb.x)[i, :], (comb.lbd)[i], (comb.is_real)[i]

def display_all_real(comb)  :
  return pd.DataFrame(
      index=np.where(comb.is_real)[0],
       data=np.concatenate([comb.lbd[np.where(comb.is_real)[0]].real[:, None], comb.x[np.where(comb.is_real)[0], :].real], axis=1),
       columns =['lbd']+ [str(i) for i in range(comb.x.shape[1])])

def display_all_complex(comb)  :
  return pd.DataFrame(
      index=np.where(comb.is_real==False)[0],
       data=np.concatenate([comb.lbd[np.where(comb.is_real==False)[0]].real[:, None], comb.x[np.where(comb.is_real==False)[0], :]], axis=1),
       columns =['lbd']+ [str(i) for i in range(comb.x.shape[1])])
    


# Example 4.3 in [Cui et al. 2014]
CUI , C.-F., DAI , Y.-H. & NIE , J. (2014) All real eigenvalues of symmetric tensors. SIAM Journal on MatrixAnalysis and Applications, 35, 1582–1601.

Generate the tensor, compare the tensor and the polynomial, then run

In [6]:
import sympy as sp
n = 3
m = 4
x0, x1, x2 = sp.symbols('x0 x1 x2')
XX = sp.Matrix([x0, x1, x2])
a = 3
P = 2*x0**4 + 3*x1**4 + 5*x2**4 + 4*a*x0**2*x1*x2
T = utils.generate_symmetric_tensor_from_poly(XX, P)
X = np.random.randn(3)
print(P.subs([(XX[i], X[i]) for i in range(3)]))
print(symmetric_tv_mode_product(T, X, 4))


4.08242003053120
4.082420030531199


In [7]:
B = np.zeros(m*[n])
for i in range(n):
  B[tuple(m*[i])] = 1

n_eig = complex_eigen_cnt(n, m, m)*3//2
print("expecting  %d eigenvalues" % complex_eigen_cnt(n, m, m))
all_eig = SimpleNamespace(
    lbd=np.full((n_eig), np.nan, dtype=complex),
    x=np.full((n_eig, n), np.nan, dtype=complex),
    is_self_conj=zeros((n_eig), dtype=bool),
    is_real=zeros((n_eig), dtype=bool))

all_eig, jcnt = find_all_unitary_eigenpair_same_order(
        all_eig, 0, T, B, max_itr=200, max_test=int(1e3), tol=1e-6, disc=3e-3)  


check_eig(T, B, all_eig)
display_all_real(all_eig).sort_values(by='lbd', ascending=False)

expecting  27 eigenvalues
Found 1 eigenpairs
Found 2 eigenpairs
Found 4 eigenpairs
Found 6 eigenpairs
Found 8 eigenpairs
Found 9 eigenpairs
Found 10 eigenpairs
Found 12 eigenpairs
Found 14 eigenpairs
Found 15 eigenpairs
Found 16 eigenpairs
Found 18 eigenpairs
Found 19 eigenpairs
check lbd 0.000000
check equation 0.000000
check uniqueness
[]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: ComplexWarning: Casting complex values to real discards the imaginary part
  # Remove the CWD from sys.path while we load stuff.


,lbd,0,1,2
8,7.450520,-5.936946e-01,-5.251851e-01,-6.096780e-01
18,7.450520,5.936946e-01,-5.251851e-01,-6.096780e-01
0,5.000000,-5.646401e-15,-9.178098e-07,-1.000000e+00
1,3.000000,-3.391020e-07,-1.000000e+00,3.382593e-07
15,2.000000,-1.000000e+00,1.292470e-26,-1.938705e-26
14,-1.395156,6.841141e-01,5.393149e-01,-4.910472e-01
9,-1.395156,6.841141e-01,-5.393149e-01,4.910472e-01


In [8]:
max_run = all_eig.lbd.shape[0]
diff3 = np.empty(max_run-1)
for i in range(max_run-1):
    factors = np.sum(all_eig.x[i, :].conj()*all_eig.x[i+1, :])
    diff3[i] = np.abs(np.abs(factors)-1)
# print(np.argsort(diff3))
print("check uniqueness")
# print(np.sort(diff3[np.where(diff3< 1e-1)]))
# np.abs(all_eig.x[7:, :]@all_eig.x[6, :].conj()[:, None])
all_eig.x[6, :].conj()


check uniqueness


array([-0.6381718 -0.10109476j, -0.5342255 -0.11802636j,
       -0.17309274+0.50321804j])

# Example 4.10 Motzkin polynomial.
We do not find the full number $n(m-1)^{n-1)$. Apply a perturbation then compute eigen pairs again

In [9]:
n = 3
m = 6
x0, x1, x2 = sp.symbols('x0 x1 x2')
XX = sp.Matrix([x0, x1, x2])

P = x0**4*x1**2 + x0**2*x1**4 + x2**6 - 3*x0**2*x1**2*x2**2
T = utils.generate_symmetric_tensor_from_poly(XX, P)
X = np.random.randn(3)
print(P.subs([(XX[i], X[i]) for i in range(n)]))
print(symmetric_tv_mode_product(T, X, m))


0.183079580474951
0.18307958047495096


In [10]:
m = 6
n = 3
B = np.zeros(m*[n])
for i in range(n):
  B[tuple(m*[i])] = 1

n_eig = complex_eigen_cnt(n, m, m)*3//2
print("expecting  %d eigenvalues" % complex_eigen_cnt(n, m, m))
all_eig = SimpleNamespace(
    lbd=np.full((n_eig), np.nan, dtype=complex),
    x=np.full((n_eig, n), np.nan, dtype=complex),
    is_self_conj=zeros((n_eig), dtype=bool),
    is_real=zeros((n_eig), dtype=bool))

all_eig, jcnt = find_all_unitary_eigenpair_same_order(
        all_eig, 0, T, B, max_itr=200, max_test=int(1e4), tol=5e-5, disc=1e-3)  


check_eig(T, B, all_eig)
display_all_real(all_eig).sort_values(by='lbd', ascending=False)

expecting  75 eigenvalues
Found 1 eigenpairs
Found 2 eigenpairs
Found 3 eigenpairs
Found 5 eigenpairs
Found 6 eigenpairs
Found 7 eigenpairs
Found 8 eigenpairs
Found 9 eigenpairs
Found 10 eigenpairs
Found 11 eigenpairs
Found 12 eigenpairs
Found 13 eigenpairs
Found 14 eigenpairs
Found 15 eigenpairs
Found 16 eigenpairs
Found 18 eigenpairs
Found 19 eigenpairs
Found 21 eigenpairs
Found 22 eigenpairs
Found 24 eigenpairs
Found 25 eigenpairs
Found 27 eigenpairs
Found 29 eigenpairs
Found 31 eigenpairs
Found 33 eigenpairs
Found 35 eigenpairs
Found 37 eigenpairs
Found 39 eigenpairs
Found 41 eigenpairs
Found 43 eigenpairs
Found 45 eigenpairs
Found 47 eigenpairs
Found 49 eigenpairs
Found 51 eigenpairs
check lbd 0.000000
check equation 0.000000
check uniqueness
[0.04534102 0.08022381 0.08022381]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: ComplexWarning: Casting complex values to real discards the imaginary part
  # Remove the CWD from sys.path while we load stuff.


,lbd,0,1,2
6,1.000000e+00,7.071068e-01,-7.071068e-01,4.496022e-18
12,1.000000e+00,-7.071068e-01,-7.071068e-01,1.714677e-16
2,1.000000e+00,-2.695292e-06,-4.028640e-05,-1.000000e+00
10,5.554044e-02,3.525470e-01,7.718280e-01,-5.291429e-01
7,5.554044e-02,7.718280e-01,-3.525470e-01,5.291429e-01
13,5.554044e-02,-3.525470e-01,-7.718280e-01,-5.291429e-01
14,5.554044e-02,7.718280e-01,3.525470e-01,-5.291429e-01
15,5.554044e-02,3.525470e-01,-7.718280e-01,5.291429e-01
9,5.554044e-02,7.718280e-01,3.525470e-01,5.291429e-01
11,5.554044e-02,-7.718280e-01,3.525470e-01,5.291429e-01


Apply a perturbation

In [11]:
Ta = T.copy()

eps = 1e-4
for i in range(n):
  Ta[m*[i]] += (i+1)*eps

# eps = 1e-3
# np.random.seed(0)
# Ta = T + eps*np.random.randn(*T.shape)
all_eig2 = SimpleNamespace(
    lbd=np.full((n_eig), np.nan, dtype=complex),
    x=np.full((n_eig, n), np.nan, dtype=complex),
    is_self_conj=zeros((n_eig), dtype=bool),
    is_real=zeros((n_eig), dtype=bool))
print("expecting  %d eigenvalues" % complex_eigen_cnt(n, m, m))
all_eig2, _ = find_all_unitary_eigenpair_same_order(
        all_eig2, 0, Ta, B, max_itr=200, max_test=int(1e4), tol=1e-6, disc=1e-3)  


check_eig(Ta, B, all_eig2)
display_all_real(all_eig2).sort_values(by='lbd', ascending=False)

expecting  75 eigenvalues
Found 1 eigenpairs
Found 2 eigenpairs
Found 4 eigenpairs
Found 5 eigenpairs
Found 7 eigenpairs
Found 8 eigenpairs
Found 10 eigenpairs
Found 12 eigenpairs
Found 14 eigenpairs
Found 15 eigenpairs
Found 17 eigenpairs
Found 19 eigenpairs
Found 20 eigenpairs
Found 22 eigenpairs
Found 24 eigenpairs
Found 26 eigenpairs
Found 27 eigenpairs
Found 28 eigenpairs
Found 29 eigenpairs
Found 30 eigenpairs
Found 32 eigenpairs
Found 34 eigenpairs
Found 36 eigenpairs
Found 37 eigenpairs
Found 38 eigenpairs
Found 40 eigenpairs
Found 41 eigenpairs
Found 42 eigenpairs
Found 44 eigenpairs
Found 45 eigenpairs
Found 47 eigenpairs
Found 49 eigenpairs
Found 50 eigenpairs
Found 52 eigenpairs
Found 54 eigenpairs
Found 56 eigenpairs
Found 58 eigenpairs
Found 60 eigenpairs
Found 61 eigenpairs
Found 62 eigenpairs
Found 63 eigenpairs
Found 65 eigenpairs
Found 67 eigenpairs
Found 69 eigenpairs
Found 71 eigenpairs
Found 73 eigenpairs
Found 75 eigenpairs
check lbd 0.000000
check equation 0.0000

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: ComplexWarning: Casting complex values to real discards the imaginary part
  # Remove the CWD from sys.path while we load stuff.


,lbd,0,1,2
27,1.004821e+00,0.706873,0.707306,6.956673e-03
0,1.000839e+00,0.005587,0.217756,9.759873e-01
29,1.000734e+00,-0.180332,-0.014617,-9.834972e-01
19,1.000553e+00,0.091930,0.044452,9.947728e-01
41,1.000000e+00,0.707107,-0.707107,-5.169879e-26
36,7.660624e-02,0.391520,0.745828,5.389368e-01
14,7.397570e-02,0.729079,0.411894,5.466145e-01
49,5.644305e-02,0.353312,-0.771585,-5.289871e-01
37,5.607954e-02,-0.772114,0.352678,-5.286374e-01
4,5.555024e-02,-0.352433,-0.771875,5.291503e-01


In [12]:
all_eig3 = SimpleNamespace(
    lbd=np.full((n_eig), np.nan, dtype=complex),
    x=np.full((n_eig, n), np.nan, dtype=complex),
    is_self_conj=zeros((n_eig), dtype=bool),
    is_real=zeros((n_eig), dtype=bool))
eig_cnt = 0
for j in range(all_eig2.lbd.shape[0]):
  bad = False
  if not np.isnan(all_eig2.lbd[j]):
    try:
      x, lbd, ctr, converge, err = schur_form_B_tensor_rayleigh_unitary_same_order(
                        T, B, max_itr=200, delta=5e-5, x_init=all_eig2.x[j, :])

    except Exception as e:
      print(e)
      bad = True
    if not bad:
      old_eig = eig_cnt
      if (err < 1e-6):
          if j == 2:
            print("before")
            print(x, lbd, ctr, converge, err, eig_cnt)
          eig_cnt = _insert_eigen(all_eig3, x, lbd, eig_cnt, m, m, tol=1e-6, disc=1e-3)
          if j == 2:
            print("after")
            print(eig_cnt)
            # break
          if (eig_cnt > old_eig):
              print('Found %d eigenpairs' % eig_cnt)
                  

Found 1 eigenpairs
Found 2 eigenpairs
before
[ 6.08474327e-05+2.87922257e-05j  3.04872529e-05+1.39628768e-05j
 -2.24046331e-01-9.74578492e-01j] (1+0j) 17 True 1.6975861756215098e-13 2
after
2
Found 3 eigenpairs
Found 4 eigenpairs
Found 5 eigenpairs
Found 7 eigenpairs
Found 9 eigenpairs
Found 10 eigenpairs
Found 11 eigenpairs
Found 12 eigenpairs
Found 14 eigenpairs
Found 16 eigenpairs
Found 17 eigenpairs
Found 18 eigenpairs
Found 19 eigenpairs
Found 20 eigenpairs
Found 21 eigenpairs
Found 23 eigenpairs
Found 25 eigenpairs
Found 26 eigenpairs
Found 28 eigenpairs
Found 30 eigenpairs
Found 32 eigenpairs
Found 34 eigenpairs
Found 36 eigenpairs
Found 37 eigenpairs
Found 38 eigenpairs
Found 39 eigenpairs
Found 41 eigenpairs
Found 43 eigenpairs
Found 45 eigenpairs
Found 47 eigenpairs
Found 49 eigenpairs
Found 51 eigenpairs


In [13]:
display_all_real(all_eig3).sort_values(by='lbd', ascending=False)

,lbd,0,1,2
0,1.000000e+00,8.455099e-16,1.724137e-04,1.000000e+00
19,1.000000e+00,7.071068e-01,-7.071068e-01,0.000000e+00
10,1.000000e+00,7.071068e-01,7.071068e-01,1.006445e-17
4,5.554044e-02,7.718280e-01,3.525470e-01,5.291429e-01
11,5.554044e-02,-7.718280e-01,-3.525470e-01,5.291429e-01
2,5.554044e-02,-3.525470e-01,-7.718280e-01,5.291429e-01
3,5.554044e-02,-3.525470e-01,7.718280e-01,-5.291429e-01
20,5.554044e-02,7.718280e-01,-3.525470e-01,-5.291429e-01
25,5.554044e-02,3.525470e-01,-7.718280e-01,-5.291429e-01
17,5.554044e-02,-7.718280e-01,3.525470e-01,-5.291429e-01


Thus, it seems there are 51 distinct pairs, and there are a number of pairs with multiplicity $>1$, similar to the Z-Pair case.

# check the eigenpair count. We let the search run for sometime, but the number of pairs found does not continue after the count $n(m-1)^{n-1}$

In [14]:
n = 3
m = 5
T = utils.generate_symmetric_tensor(n, m)
B = np.zeros(m*[n])
for i in range(n):
  B[tuple(m*[i])] = 1

n_eig = complex_eigen_cnt(n, m, m)*3//2
print("expecting  %d eigenvalues" % complex_eigen_cnt(n, m, m))
all_eig = SimpleNamespace(
    lbd=np.full((n_eig), np.nan, dtype=complex),
    x=np.full((n_eig, n), np.nan, dtype=complex),
    is_self_conj=zeros((n_eig), dtype=bool),
    is_real=zeros((n_eig), dtype=bool))

all_eig, jcnt = find_all_unitary_eigenpair_same_order(
        all_eig, 0, T, B, max_itr=200, max_test=int(1e4), tol=1e-6, disc=1e-3)  


check_eig(T, B, all_eig)
display_all_real(all_eig).sort_values(by='lbd', ascending=False)

expecting  48 eigenvalues
Found 2 eigenpairs
Found 3 eigenpairs
Found 5 eigenpairs
Found 6 eigenpairs
Found 8 eigenpairs
Found 10 eigenpairs
Found 12 eigenpairs
Found 13 eigenpairs
Found 15 eigenpairs
Found 17 eigenpairs
Found 19 eigenpairs
Found 21 eigenpairs
Found 23 eigenpairs
Found 24 eigenpairs
Found 26 eigenpairs
Found 28 eigenpairs
Found 30 eigenpairs
Found 32 eigenpairs
Found 34 eigenpairs
Found 36 eigenpairs
Found 38 eigenpairs
Found 40 eigenpairs
Found 42 eigenpairs
Found 44 eigenpairs
Found 46 eigenpairs
Found 48 eigenpairs
check lbd 0.000000
check equation 0.000000
check uniqueness
[0.06101182 0.06938987]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: ComplexWarning: Casting complex values to real discards the imaginary part
  # Remove the CWD from sys.path while we load stuff.


,lbd,0,1,2
23,51.734268,0.590192,0.576622,0.564961
5,5.082689,-0.627191,0.325009,-0.707814
2,3.578579,0.293646,-0.624370,-0.723833
12,1.525379,0.785608,0.484634,-0.384642


In [15]:
n = 5
m = 3
T = utils.generate_symmetric_tensor(n, m)
B = np.zeros(m*[n])
for i in range(n):
  B[tuple(m*[i])] = 1

n_eig = complex_eigen_cnt(n, m, m)*3//2
print("expecting  %d eigenvalues" % complex_eigen_cnt(n, m, m))
all_eig = SimpleNamespace(
    lbd=np.full((n_eig), np.nan, dtype=complex),
    x=np.full((n_eig, n), np.nan, dtype=complex),
    is_self_conj=zeros((n_eig), dtype=bool),
    is_real=zeros((n_eig), dtype=bool))

all_eig, jcnt = find_all_unitary_eigenpair_same_order(
        all_eig, 0, T, B, max_itr=200, max_test=int(1e4), tol=1e-6, disc=1e-3)  


expecting  80 eigenvalues
Found 1 eigenpairs
Found 3 eigenpairs
Found 4 eigenpairs
Found 6 eigenpairs
Found 8 eigenpairs
Found 10 eigenpairs
Found 12 eigenpairs
Found 14 eigenpairs
Found 15 eigenpairs
Found 16 eigenpairs
Found 18 eigenpairs
Found 20 eigenpairs
Found 22 eigenpairs
Found 24 eigenpairs
Found 26 eigenpairs
Found 28 eigenpairs
Found 30 eigenpairs
Found 32 eigenpairs
Found 34 eigenpairs
Found 36 eigenpairs
Found 37 eigenpairs
Found 39 eigenpairs
Found 41 eigenpairs
Found 42 eigenpairs
Found 44 eigenpairs
Found 46 eigenpairs
Found 47 eigenpairs
Found 49 eigenpairs
Found 51 eigenpairs
Found 52 eigenpairs
Found 54 eigenpairs
Found 56 eigenpairs
Found 58 eigenpairs
Found 60 eigenpairs
Found 61 eigenpairs
Found 62 eigenpairs
Found 64 eigenpairs
Found 66 eigenpairs
Found 68 eigenpairs
Found 70 eigenpairs
Found 71 eigenpairs
Found 73 eigenpairs
Found 74 eigenpairs
Found 76 eigenpairs
Found 78 eigenpairs
Found 79 eigenpairs
Found 80 eigenpairs


In [16]:
n = 4
m = 5
np.random.seed(0)

T = utils.generate_symmetric_tensor(n, m)
B = np.zeros(m*[n])
for i in range(n):
  B[tuple(m*[i])] = 1

n_eig = complex_eigen_cnt(n, m, m)*3//2
print("expecting  %d eigenvalues" % complex_eigen_cnt(n, m, m))
all_eig = SimpleNamespace(
    lbd=np.full((n_eig), np.nan, dtype=complex),
    x=np.full((n_eig, n), np.nan, dtype=complex),
    is_self_conj=zeros((n_eig), dtype=bool),
    is_real=zeros((n_eig), dtype=bool))

all_eig, jcnt = find_all_unitary_eigenpair_same_order(
        all_eig, 0, T, B, max_itr=200, max_test=int(1e5), tol=1e-6, disc=1e-3)  


expecting  256 eigenvalues
Found 2 eigenpairs
Found 4 eigenpairs
Found 5 eigenpairs
Found 6 eigenpairs
Found 8 eigenpairs
Found 10 eigenpairs
Found 12 eigenpairs
Found 13 eigenpairs
Found 15 eigenpairs
Found 17 eigenpairs
Found 19 eigenpairs
Found 20 eigenpairs
Found 21 eigenpairs
Found 23 eigenpairs
Found 24 eigenpairs
Found 25 eigenpairs
Found 26 eigenpairs
Found 28 eigenpairs
Found 30 eigenpairs
Found 32 eigenpairs
Found 33 eigenpairs
Found 35 eigenpairs
Found 37 eigenpairs
Found 39 eigenpairs
Found 41 eigenpairs
Found 43 eigenpairs
Found 45 eigenpairs
Found 46 eigenpairs
Found 48 eigenpairs
Found 50 eigenpairs
Found 52 eigenpairs
Found 54 eigenpairs
Found 56 eigenpairs
Found 58 eigenpairs
Found 60 eigenpairs
Found 62 eigenpairs
Found 64 eigenpairs
Found 66 eigenpairs
Found 68 eigenpairs
Found 70 eigenpairs
Found 72 eigenpairs
Found 74 eigenpairs
Found 76 eigenpairs
Found 78 eigenpairs
Found 80 eigenpairs
Found 82 eigenpairs
Found 84 eigenpairs
Found 86 eigenpairs
Found 88 eigenpair